In [3]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import SnowballStemmer
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib as plt
%matplotlib inline

import os

In [4]:
_, _, blog_file_names = next(os.walk("./datasets/blogs/"))

In [14]:
def clear_html_entities(input_file_path, output_file_path):
    with open(input_file_path, 'r', encoding='latin-1') as f:
        try:
            text = f.read()
            text = text.encode('ascii', 'ignore').decode('ascii')
        except UnicodeDecodeError as e:
            print(input_file_path)
            raise e

    text = re.sub(r'[\x00-\x09\x0B\x0C\x0E-\x1F\x7F]', '', text) # Remove non-printable characters
    text = re.sub(r'&.*?;', ' ', text)
    text = re.sub(r"&", "", text)
    text = re.sub('<(?![a-zA-Z/])', '&lt;', text)
    text = re.sub('</3', '', text)
    text = text.replace('<3', '&lt;3')
    text = text.replace('<>', ' ')
    
    with open(output_file_path, 'w') as f:
        f.write(text)

In [6]:
import xml.etree.ElementTree as ET

def build_dict_from_xml(file_path):
    # Parse the XML file and get the root element
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Initialize an empty dictionary
    posts_by_date = {}

    # Initialize a variable to hold the current date
    current_date = None

    # Iterate over the children of the root element
    for child in root:
        if child.tag == "date":
            # Store the date and initialize a new list of posts
            current_date = child.text
            posts_by_date[current_date] = []
        elif child.tag == "post":
            # Append the post to the list of posts for the current date
            posts_by_date[current_date].append(child.text.strip())

    return posts_by_date

In [42]:
clear_html_entities("./datasets/blogs/1000866.female.17.Student.Libra.xml", "/tmp/blog-text.txt")
# print(build_dict_from_xml("./datasets/blogs/1000866.female.17.Student.Libra.xml"))

In [30]:
print(build_dict_from_xml("/tmp/blog-text.txt"))

{'23,November,2002': ["Yeah, sorry for not writing for a whole there, but I've had a pretty busy weekend so far. I found out I have about 20 tests on Tuesday because my teachers are evil...but oh well. We had marching yesterday. It was cool. We stayed inside the whole time, and I got to play second part, so the music part will be really easy. I left early from marching at around 5:00, then went out to dinner with my friends (I made the plans before I knew about marching). So, dinner was fun. It was Kelly's birthday, so it was cool. Then we went to Kel's for a sleepover, and did bunches of neat stuff, and I went to sleep earlier than usual for a sleepover since I had to be at a Bar Mitzvah this morning. It was pretty neat. I have to do this project for CR, and we have to go to two different places of worship than our own. So, one is obviously a Jewish synagogue, and I think I'm going to go to a mosque for my other one, but who knows...so, this has been my only chance to write, and I act

In [16]:
df = pd.DataFrame(columns=['gender', 'age', 'date', 'post'])

for blog_file in blog_file_names:
    gender = blog_file.split('.')[1]
    age = blog_file.split('.')[2]

    file_url = "./datasets/blogs/" + blog_file
    clear_html_entities(file_url, "/tmp/blog-text.txt")
    posts_by_date = build_dict_from_xml("/tmp/blog-text.txt")

    for date in posts_by_date:
        for post in posts_by_date[date]:
            new_row = pd.DataFrame({'date': [date], 'gender': [gender], 'age': [age], 'post': [post]})
            df = pd.concat([df, new_row], ignore_index=True)

ParseError: not well-formed (invalid token): line 3572, column 0 (<string>)